# COMS 4995_002 Deep Learning Assignment 2
Due on Thursday, Feb 27, 11:59pm

This assignment can be done in groups of at most 2 students. Everyone must submit on Courseworks individually.

Member 1: Michal Porubcin, mp3242

Member 2: Daniel Echlin, dje2126

In [2]:
import numpy as np
import pandas as pd
import scipy.misc
import glob
import sys
import random
import tensorflow as tf

In [ ]:
#### CNN Code here ####

In [ ]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)


def resize_img(img_arr, target_dim=(50, 50)):
    """
    Resizes img represented as numpy array
    """
    return scipy.misc.imresize(img_arr, target_dim)


def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [ ]:
# Functions to load data

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path, suffix="train"):
    """
    Return X and y
    """
    train_data_path = data_root_path + suffix
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    np.save(filename, y)

In [ ]:
# Load train data
data_root_path = './cifar10-hw2/'
# data_root_path = './cifar10-simple/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_centered = (X_train - np.mean(X_train, axis=1)[:, np.newaxis])
X_div = (X_centered / np.std(X_centered, axis=1)[:, np.newaxis])
X_train = X_div
print('Train data loading done')

In [ ]:
#Load test data
X_test = get_images(data_root_path + 'test')
X_centered = (X_test - np.mean(X_test, axis=1)[:, np.newaxis])
X_div = (X_centered / np.std(X_centered, axis=1)[:, np.newaxis])
X_test = X_div
print('Test data loading done')

In [ ]:
#Run CNN
layer_dimensions = [X_train.shape[0], 500, 500, 500, 10]
#CNN = SuperCNN(layer_dimensions, drop_prob=0)
#CNN.train(X_train, y_train, iters=2500, alpha=0.0005, batch_size=200)

In [33]:
#Predict
y_predicted, cache = NN.predict(X_test)
save_predictions('ans1-mp3242', y_predicted)

In [ ]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-mp3242.npy')
print(loaded_y.shape)
loaded_y[:10]

# Write down results here:

Below is our setup:
- Hidden Layers: [1536]
- alpha: .0005, decay factor: .83 (manually set on line 384)
- Batch size: 200
- 2500 iterations

We got about xx% validation accuracy with this run.

Blablabla